In [ ]:
#===========================================================================================
# ElasticSearch 텍스트 임베딩 테스트 예제
# - synap 문서추출 필터를 이용해 추출한 문서들을 문장클러스터링 임베딩 10개를 생성하여 ES에 추가하는 예제
# - synap 문서 추출해서 임베딩 벡터 생성하는 예시는 https://github.com/kobongsoo/BERT/blob/master/synap/mpower-sbert-Faiss-MRR-sentence-clustering.ipynb 참조
#
# -여기서는 elasticsearch 7.17.3 때를 기준으로 설명함.
# -** 따라서 elasticsearch python 모듈도 7.17.3 을 설치해야 함
# - elasticsearch 모듈 8.x 부터는 구문의 많이 변경되었음.
# - 예 : index 생성:  body로 모든 변수들를 지정하는 데시, 명시적으로 모든 변수들을 최상으로 지정해 줘야함.
# => 참고: https://towardsdatascience.com/important-syntax-updates-of-elasticsearch-8-in-python-4423c5938b17   

# =>ElasticSearch 7.3.0 버전부터는 cosine similarity 검색을 지원한다.
# => 데이터로 고차원벡터를 집어넣고, 벡터형식의 데이터를 쿼리(검색어)로 하여 코사인 유사도를 측정하여 가장 유사한 데이터를 찾는다.
# => 여기서는 ElasticSearch와 S-BERT를 이용함
# => ElasticSearch에 index 파일은 index_1.json /데이터 파일은 KorQuAD_v1.0_train_convert.json 참조
#
# => 참고자료 : https://skagh.tistory.com/32
#
#===========================================================================================

# sentenceTransformers 라이브러리 설치
#!pip install -U sentence-transformers

# elasticsearch 서버 접속 모듈 설치
# !pip install elasticsearch==7.17

# 한국어 문장 분리기(kss) 설치
#!pip install kss

# 추출 요약 설치
#!pip install bert-extractive-summarizer

In [ ]:
    
import os
import platform

# os가 윈도우면 from eunjeon import Mecab 
if platform.system() == 'Windows':
    os.environ["OMP_NUM_THREADS"] = '1' # 윈도우 환경에서는 쓰레드 1개로 지정함

import torch
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.cross_encoder import CrossEncoder

import kss
import numpy as np
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from tqdm.notebook import tqdm

from elasticsearch import Elasticsearch
from elasticsearch import helpers


# FutureWarning 제거
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 

import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging, getListOfFiles

logger = mlogging(loggername="synap", logfilename="../../log/synap")
device = GPU_info()
#device = torch.device('cpu')

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
seed = 111
query_num = 0               # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
search_k = 5                # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
avg_num = 1                 # 쿼리에 대해 sub 문장들중 최대 scorce를 갖는 문장을 몇개 찾고 평균낼지.(3=쿼리에 가장 유사한 sub문장 3개를 찾고 평균을 냄)
faiss_index_method = 1      # 0= Cosine Similarity 적용(IndexFlatIP 사용), 1= Euclidean Distance 적용(IndexFlatL2 사용)

# 임베딩 방식 (0=문장클러스터링, 1=문장평균임베딩, 2=문장임베딩)
EMBEDDING_METHOD = 0    
FLOAT_TYPE = 'float16'     # 'float32' 혹은 'float16' => 모델임베딩후 출력되는 벡터 float 타입을 지정.=>단 FAISS에 인덱스할때는 'float32'만 지원하므로, .astype('float32')로 형변환 해줘야 함.

# 청크 분할 혹은 슬라이딩 윈도우param
IS_SLIDING_WINDOW = False
WINDOW_SIZE=256             # 문단을 몇 token으로 나눌지          (128,0)=>78.40%, (256, 64)=>75.20%
SLIDING_SIZE=0              # 중첩되는 token 

# 클러스트링 param
CLUSTRING_MODE = "kmeans"  # "kmeans" = k-평균 군집 분석, kmedoids =  k-대표값 군집 분석
num_clusters = 10           # 클러스터링 계수 
outmode = "mean"           # 클러스터링후 출력벡터 정의(kmeans 일때 => mean=평균벡터 출력, max=최대값벡터출력 / kmedoids 일때=>mean=평균벡터, medoid=대표값벡터)

# ONNX 모델 사용 유.무
IS_ONNX_MODEL = False

# 차원 축소
out_dimension = 128  # 768은 0으로 입력, 128=128 입력
if out_dimension == 0:
    dimension = 768

# 문장 전처리
remove_sentence_len = 8    # 문장 길이가 10이하면 제거 
remove_duplication = False  # 중복된 문장 제거(*중복된 문장 제거 안할때 1%정도 정확도 좋음)

use_bm25 = False           # BM25 출력 할지=True. 안할지=False
#------------------------------------------------------------------------------------

assert FLOAT_TYPE == 'float16' or FLOAT_TYPE == 'float32', f'FLOAT_TYPE은 float16, float32 만 입력가능합니다. 현재 입력 FLOAT_TYPE={FLOAT_TYPE}'
assert CLUSTRING_MODE == 'kmeans' or CLUSTRING_MODE == 'kmedoids', f'CLUSTRING_MODE는 kmeans, kmedoids 만 입력가능합니다. 현재 입력 CLUSTRING_MODE={CLUSTRING_MODE}'
assert EMBEDDING_METHOD == 0 or EMBEDDING_METHOD == 1 or EMBEDDING_METHOD == 2, f'EMBEDDING_METHOD는  0,1,2 만 입력가능합니다. 현재 입력 EMBEDDING_METHOD={EMBEDDING_METHOD}'
assert out_dimension == 0 or out_dimension == 128, f'out_dimension는  0,128 만 입력가능합니다. 현재 입력 out_dimension={out_dimension}'

seed_everything(seed)

# elastic 서버 접속 테스트
#es = Elasticsearch("https://192.168.0.27:9200/", verify_certs=False)
#es = Elasticsearch("http://192.168.0.27:9200/")
#es.info()


In [ ]:
# 문장 분리 테스트
import kss
import time
from myutils import split_sentences1,split_sentences

start = time.time()

contexts = [
'''
클라우드서비스 구축 신청서
신청인
업체명 : 
사업자등록번호 : 
주소 : 
대표자
전화번호 : 
서비스
총괄 실무자
성명 : 
전화번호 : 
이메일 : 
클라우드서비스 구축 기간
※ 최대 6개월 이내
클라우드서비스의 구분
[ ] SaaS (Software as a Service) ( □ 간편등급 □ 표준등급 )
※ 인증받은 IaaS 서비스명 : 
※ SaaS 보안인증 신청 시 중요자료를 다루는 전자결재, 회계관리, 인적자원관리, 보안서비스, PaaS 5개 분야를 제외하고는 모든 서비스 간편등급 신청이 가능
클라우드서비스 명칭
※ 구축하고자 하는 SaaS 서비스명
클라우드서비스에 대한
간략한 설명
※ 구축될 클라우드서비스의 활용 분야(통신, 금융 등) 및 주요 기능 (이메일, Web Security)등 설명
동의사항
1. 본 시스템(서비스) 구축은 KISA의 클라우드 보안인증 수검을 위한 것으로, 다른 용도로 사용하지 않을 것을 서약하며, 본 용도 외 목적으로 사용중 발생하는 각종 문제에 대한 책임은 본 신청사에 있다. 
2. 구축 및 테스트를 위한 임시 사용 기간이 최대 6개월임을 인지하였으며, 연장시, 구축을 재신청하여야 함. 기간이 경과하였음에도 별도 재신청을 하지 않을 경우, IaaS 사업자는 자원을 회수하거나 접근 계정 삭제 등의 조치를 취할 수 있다. 
3. 신청사는 적절한 보안수준의 유지를 위하여 노력하여야 하며 각종 장애로 인하여 정상적인 서비스가 어려운 경우에 이를 신속하게 수리 및 복구하기 위해 성실히 협조한다. 
위와 같이 클라우드 보안인증 취득 준비를 위한 클라우드서비스 구축을 진행하고자 합니다.
년 월 일
신청인(대표자) 
(서명 또는 인)
'''
]  

doc_sentences = split_sentences1(paragraphs=contexts, 
                                    remove_line=False, 
                                    remove_sentence_len=8, 
                                    remove_duplication=False, 
                                    check_en_ko=False, # 한국어 혹은 영어문장이외 제거하면, 즉 true 지정하면 1% 성능 저하됨
                                    sentences_split_num=10000, paragraphs_num=10000000, showprogressbar=True, debug=False)

print(len(doc_sentences[0]))

print()
print(f'*문장처리=>len:{len(doc_sentences[0])}, time:{time.time()-start:.4f}')

for doc_sentence in doc_sentences[0]:
    print(doc_sentence)


In [ ]:
# 엘라스틱 서치에 인덱스 생성 하고 dense 벡터 128차원 5개 추가하는 예제임
'''
from elasticsearch import Elasticsearch
import numpy as np

# define the Elasticsearch client
es = Elasticsearch("http://192.168.0.27:9200/")

mapping='''
{
  "settings": {
    "number_of_shards": 2,
    "number_of_replicas": 1
  },
   "mappings": {
    "dynamic": "true",
    "_source": {
      "enabled": "true"
    },
    "properties": {
      "vector1": {
        "type": "dense_vector",
        "dims": 128
      },
	  "vector2": {
        "type": "dense_vector",
        "dims": 128
      },
      "vector3": {
        "type": "dense_vector",
        "dims": 128
      },
      "vector4": {
        "type": "dense_vector",
        "dims": 128
      },
      "vector5": {
        "type": "dense_vector",
        "dims": 128
      }
    }
  }
}
'''

###########################################################
# 인덱스 생성/삭제
###########################################################
## 인덱스 생성
def create_index(index, mapping=None):
    if not es.indices.exists(index=index):
        return es.indices.create(index=index ,body=mapping)
    
## 인덱스 자체 삭제
def delete_index(index):
    if es.indices.exists(index=index):
        return es.indices.delete(index=index)
    
###########################################################
# 인데스에 데이터 추가 
###########################################################
def insert(index, doc_type, body):
    return es.index(index=index, doc_type=doc_type, body=body)

# index 생성
INDEX_NAME = 'my_index_128_5-1'
res=create_index(index=INDEX_NAME, mapping=mapping)
print(res)
print('\n')

result=es.indices.get_settings(index=INDEX_NAME)
print(result)
print('\n')


# document 추가 
DOC_TYPE = '_doc'

# define the dense vector
dense_vector1 = np.random.rand(128).tolist()
dense_vector2 = np.random.rand(128).tolist()
dense_vector3 = np.random.rand(128).tolist()
dense_vector4 = np.random.rand(128).tolist()
dense_vector5 = np.random.rand(128).tolist()

doc = {
    'vector1': dense_vector1,
    'vector2': dense_vector2,
    'vector3': dense_vector3,
    'vector4': dense_vector4,
    'vector5': dense_vector5
}

res = insert(index=INDEX_NAME, doc_type=DOC_TYPE, body=doc) 
print(res)
'''

In [ ]:
#-------------------------------------------------------------------------------------
# 1. 검색모델 로딩
# => bi_encoder 모델 로딩, polling_mode 설정
# => bi_encoder1 = SentenceTransformer(bi_encoder_path) # 오히려 성능 떨어짐. 이유는 do_lower_case나, max_seq_len등 세부 설정이 안되므로.
#-------------------------------------------------------------------------------------
import torch
from myutils import bi_encoder, dense_model, onnx_model, onnx_embed_text
from sentence_transformers import SentenceTransformer

#bi_encoder_path = "../../data11/model/bert/moco-sentencebertV2.0-nli_128d-sts" 
bi_encoder_path = "../../data11/model/kpf-sbert-128d-v1" #"bongsoo/kpf-sbert-v1.1" # kpf-sbert-v1.1 # klue-sbert-v1 # albert-small-kor-sbert-v1.1
pooling_mode = 'mean' # bert면=mean, albert면 = cls


word_embedding_model1, bi_encoder1 = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, 
                                                pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
  
print(f'\n---bi_encoder---------------------------')
print(bi_encoder1)
print(word_embedding_model1)
#------------------------------------------------------------------------------------------------------------------------

In [ ]:
from myutils import embed_text, onnx_embed_text

# 조건에 맞게 임베딩 처리하는 함수 
def embedding(paragrphs:list):
    if IS_ONNX_MODEL == True:
        embeddings = onnx_embed_text(model=onnx_model, tokenizer=onnx_tokenizer, paragraphs=paragrphs, token_embeddings=False).astype(FLOAT_TYPE)  
    else:
        # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
        embeddings = embed_text(model=bi_encoder1, paragraphs=paragrphs, return_tensor=False).astype(FLOAT_TYPE)  
    
    return embeddings

In [ ]:
#=========================================================
# 여기서 부터는 문서 인덱싱 처리 임.
#=========================================================

In [ ]:
import os
import pandas as pd
from myutils import remove_reverse, getListOfFiles, clean_text
from tqdm.notebook import tqdm

# 문서 추출된 TEXT들을 dataframe 형태로 만듬.
OUT_FOLDER = '../../data11/mpower_doc/out/' # 추출된 TEXT 파일들이 있는 루트폴더

# OUT_FOLDER에 모든 파일 경로를 얻어옴.
file_paths = getListOfFiles(OUT_FOLDER)
assert len(file_paths) > 0 # files가 0이면 assert 발생
    
print('*file_count: {}, file_list:{}'.format(len(file_paths), file_paths[0:5]))

contexts = []
titles = []
contextids = []

# TEXT 추출된 파일들을 읽어오면서 제목(title), 내용(contexts) 등을 저장해 둠.
contextid = 1000
for idx, file_path in enumerate(tqdm(file_paths)):
    if '.ipynb_checkpoints' not in file_path:
        sentences = []
        with open(file_path, 'r', encoding='utf-8') as f:
            data = f.read()
            
            #.PAGE:1 패턴을 가지는 문장은 제거함.
            pattern = r"\.\.PAGE:\d+\s?"
            data = clean_text(text=data, pattern=pattern)
            
            file_name = os.path.basename(file_path)  # 파일명만 뽑아냄
            
            #  filename = 5.보안사업부 사업계획.hwp.txt 이면 뒤에 hwp.txt는 제거하고 '5.보안사업부 사업계획' 문자열만 title로 저장함.
            file_name = remove_reverse(file_name, '.')# 5.보안사업부 사업계획.hwp 출력됨
            file_name = remove_reverse(file_name, '.')# 5.보안사업부 사업계획 출력됨
            
            contextid += 1
            contexts.append(data)     # 파일 내용 저장 
            titles.append(file_name)  # 파일명을 제목으로 저장(추후 쿼리할 문장이 됨)
            contextids.append(contextid) # contextid 저장 
 
# 데이터 프레임으로 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])
df_questions = pd.DataFrame((zip(titles, contextids)), columns = ['question','contextid'])

print(f'*len(contexts): {len(contexts)}')

In [ ]:
df_contexts.head()

In [ ]:
df_questions.tail()

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 3. 슬라이딩 윈도우 혹은 문장 분리  
# 1) 슬라이딩 윈도우 : 문장(문서)들을 chunk(청크: 큰 덩어리)로 분리, 다시 분리된 chunks를 kss로 문장 분리해서 sentences 만듬, 이후 chunks와 sentences를 doc_sentences에 담음
#    최대 512 단위로 서로 겹치게 청크 단위로 분리함.
# 2) 문장 분리 : kss와 \n(줄바꿈)으로 문장을 분리함.
#-------------------------------------------------------------------------------------------------------
import time
from myutils import sliding_window_tokenizer, split_sentences, split_sentences1, get_text_chunks

#text = ["해외에서 데이터 무제한?? 뿐 아니라 음성과 문자 요금을 할인 받을 수 있는 ++요금제가 나왔다. 날씨는 좋다. lg유플러스는 중국, 일본, 홍콩, 싱가포르, 필리핀 등 아시아 8국을 대상으로~~ 무제한 데이터와 음성, 문자를 할인해주는 '스마트 로밍 요금제' 2종을 오는 28일부터 판매한다고 27일 밝혔다. . . 우선 '스마트 로밍음성'은 하루 기본료가 2000원으로 음성발신은 1분당 1000원이며, 문자메시지(sms)와 멀티미디어 문자메시지(mms)는 1건당 150원이다. 종전 500원에서 350원 인하했다. . . '스마트 로밍패키지'는 여기에 데이터 무제한 로밍서비스가 더해져 하루 기본료가 1만1000원이다. 기본료는 로밍 기간에 상관없이 사용한 당일에만 청구된다. . . 스마트 로밍 요금제는 일단 오는 7월 말까지 프로모션 형태로 제공하고 이후 정식 요금제로 추진한다는 계획이다. . . 아울러 해외에서 무제한 데이터 로밍 서비스와 데이터로밍 차단을 신청·해지할 수 있는 로밍 전용 모바일 홈페이지((이메일))도 운영한다. . . (이름) lg유플러스 글로벌로밍팀장은 '아시아 출(이름) 여행을 계획하고 있는 고객들이 저렴한 비용으로 안심하고 로밍 서비스를 사용할 수 있도록 이번 프로모션 요금제를 출시했다'며 '지속적으로 해외 로밍 이용 고객들의 편의성 증대를 위한상품을 준비할 예정'이라고 말했다. . . . . . . . # # #. . ■ 사진설명. . lg유플러스(부회장 (이름) / (이메일) )가 4월 28일부터 중국, 일본 등 아시아 8개국*을 대상으로 데이터와 음성 문자까지 할인해서 제공하는 스마트 로밍요금제(스마트 로밍음성/스마트 로밍패키지) 2종을 출시했다."]
#text =["오늘은 날씨가 ^^~~~~좋다. 내일은 비가 오고 춥겠다고 한다^^. 걱정이다. 오늘만큼만 매일 날씨가 좋으면 좋겠다.~~~~"]
contexts = df_contexts['context'].values.tolist()
start = time.time()


doc_sentences = []
tokenizer = word_embedding_model1.tokenizer
    
# 슬라이딩 윈도우 처리 후 chunks 리스트 만들고, 다시 chunks를 kss로 문장 분리해서, 최종 chunks와 sentences를 doc_sentences에 담음
if IS_SLIDING_WINDOW == True:
    for idx, context in enumerate(tqdm(contexts)):
        
        clean_context = clean_text(context)  # 전처리 : (한글, 숫자, 영문, (), {}, [], %, ,,.,",')  등을 제외한 특수문자 제거
        
        pattern = r'^\d+(\.\d+)*'  # 문장 맨앞에 '4.1.2.3' 패턴 제거,  r'^\d+(\.\d+)*\s+' # 문장 맨앞에 '4.1.2.3띄어쓰기' 있는 패턴 제거
        clean_context = clean_text(text=clean_context, pattern=pattern)
                        
        pattern = r'^\d+\.'  # 문장 맨 앞에 '4.' 패턴 제거
        clean_context = clean_text(text=clean_context, pattern=pattern)
                        
        pattern = r'^\d+\)'  # 문장 맨 앞에 '4)' 패턴 제거
        clean_context = clean_text(text=clean_context, pattern=pattern)
         
              
        #chunks = sliding_window_tokenizer(tokenizer = tokenizer, paragraph=clean_context, window_size=WINDOW_SIZE, sliding_size=SLIDING_SIZE)
        
        # 청크 클러스터링만 수행함
        chunks = get_text_chunks(tokenizer = tokenizer, paragraph=clean_context, chunk_token_size=WINDOW_SIZE)
        doc_sentences.append(chunks)
        
        #sentences = split_sentences1(paragraphs=chunks, remove_line=False, remove_sentence_len=remove_sentence_len, sentences_split_num=10000, paragraphs_num=10000000, debug=False)

        # chunks 1차원 리스트[A,B,C]와 sentences 2차원 리스트[[a,b],[b,c],[d,e]] 를 합쳐서, doc_sentencs[A,a,b,B,b,c,C,d,e] 에 담음.
        #arr = []
        #[arr.extend([chunks[i], *sentences[i]]) for i in range(len(chunks))]

        # doc_sentences 리스트에 추가 
        #doc_sentences.append(arr)
        
# 문장 분리해서 doc_sentences에 담음.
else:
    doc_sentences = split_sentences1(paragraphs=contexts, 
                                    remove_line=False, 
                                    remove_sentence_len=remove_sentence_len, 
                                    remove_duplication=remove_duplication, 
                                    check_en_ko=False, # 한국어 혹은 영어문장이외 제거하면, 즉 true 지정하면 1% 성능 저하됨
                                    sentences_split_num=10000, paragraphs_num=10000000, showprogressbar=True, debug=False)

logger.info(f'*문장처리=>len:{len(doc_sentences[0])}, time:{time.time()-start:.4f}')

len_list = []
for i, doc_sentence in enumerate(doc_sentences):
    doc_sentence_len = len(doc_sentence)
    if i < 301:
        print(f'[{i}] {doc_sentence_len}/{df_questions["question"][i]}')
    len_list.append(doc_sentence_len)
    
logger.info(f'*문장 길이=>평균:{sum(len_list) / len(len_list)} / MAX: {max(len_list)} / MIN: {min(len_list)}')

In [ ]:
#---------------------------------------------------------------------------
# ES 인덱스 생성
# -in : es : ElasticSearch 객체.
# -in : create : 기존에 동일한 인덱스는 삭제하고 다시 생성.
#---------------------------------------------------------------------------
def create_index(es, create:bool = True):
    
    if create == True:
        es.indices.delete(index=INDEX_NAME, ignore=[404])
        count = 0
        
        # 인덱스 생성
        with open(INDEX_FILE) as index_file:
            source = index_file.read().strip()
            count += 1
            print(f'{count}:{source}') # 인덱스 구조 출력
            es.indices.create(index=INDEX_NAME, body=source)

In [ ]:
#---------------------------------------------------------------------------
# 인덱스 batch 처리
# - in: ES 객체
# - in: docs=인덱스 처리할 data
# - in: vector_len=한문서에 인덱싱할 벡터수=클러스터링수와 동일(기본=10개)
# - in: dim_size=벡터 차원(기본=128)
#---------------------------------------------------------------------------
def index_batch(es, docs, vector_len:int=10, dim_size:int=128):
        
    requests = []
    
    for i, doc in enumerate(tqdm(docs)):
        rfile_name = doc['rfile_name']
        rfile_text = doc['rfile_text']
        dense_vectors = doc['dense_vectors']
        
        #--------------------------------------------------------------------
        # ES에 문단 인덱싱 처리
        request = {}  #dict 정의
        request["rfile_name"] = rfile_name       # 제목               
        request["rfile_text"] = rfile_text   # 문장
        
        request["_op_type"] = "index"        
        request["_index"] = INDEX_NAME
        
        # vector 1~40 까지 값을 0으로 초기화 해줌.
        for i in range(vector_len):
            request["vector"+str(i+1)] = np.zeros((dim_size))
            
        # vector 값들을 담음.
        for i, dense_vector in enumerate(dense_vectors):
            request["vector"+str(i+1)] = dense_vector
            
        requests.append(request)
        #--------------------------------------------------------------------
                
    # batch 단위로 한꺼번에 es에 데이터 insert 시킴     
    bulk(es, requests)

In [ ]:
# 분리된 문장들에 대해 클러스터링 실행
#-------------------------------------------------------------------------------------------------------
#문단에 문장들의 임베딩을 구하여 각각 클러스터링 처리함.
#-------------------------------------------------------------------------------------------------------
# Then, we perform k-means clustering using sklearn:
from sklearn.cluster import KMeans
from myutils import embed_text, fassi_index, clustering_embedding, kmedoids_clustering_embedding

def index_data():
    #클러스터링 계수는 문단의 계수보다는 커야 함. 
    #assert num_clusters <= len(doc_sentences), f"num_clusters:{num_clusters} > len(doc_sentences):{len(doc_sentences)}"
    #-------------------------------------------------------------
    # 각 문단의 문장들에 벡터를 구하고 리스트에 저장해 둠.
    start = time.time()
    cluster_list = []

    rfile_names = df_questions['contextid'].values.tolist()
    rfile_texts = df_questions['question'].values.tolist()

    docs = []
    count = 0
    for i, sentences in enumerate(tqdm(doc_sentences)):
        embeddings = embedding(sentences)
        if i < 3:
            print(f'[{i}] sentences---------------------------EMBEDDING_METHOD={EMBEDDING_METHOD}')
            print(sentences)
            print(f'embeddings.shape: {embeddings.shape}')

        # 0=문장클러스터링 임베딩
        if EMBEDDING_METHOD == 0:
            if CLUSTRING_MODE == "kmeans":
                # 각 문단에 분할한 문장들의 임베딩 값을 입력해서 클러스터링 하고 평균값을 구함.
                #emb1 = clustering_embedding(embeddings = embeddings, outmode=outmode, num_clusters= 50, seed=seed)
                emb = clustering_embedding(embeddings = embeddings, outmode=outmode, num_clusters= num_clusters, seed=seed).astype(FLOAT_TYPE) 
            else:
                emb = kmedoids_clustering_embedding(embeddings = embeddings, outmode=outmode, num_clusters= num_clusters, seed=seed).astype(FLOAT_TYPE) 
        # 1= 문장평균임베딩
        elif EMBEDDING_METHOD == 1:
            # 문장들에 대해 임베딩 값을 구하고 평균 구함.
            arr = np.array(embeddings).astype(FLOAT_TYPE)
            emb = arr.mean(axis=0).reshape(1,-1) #(128,) 배열을 (1,128) 형태로 만들기 위해 reshape 해줌
        # 2=문장임베딩
        else:
            emb = embeddings

        #emb.astype('float16')
        if i < 2:
            print(f'emb.shape: {emb.shape}')
            print(f'emb:{emb[0]}')

        #--------------------------------------------------- 
        count += 1
        doc = {} #dict 선언

        
        doc['rfile_name'] = rfile_names[i]      # contextid 담음
        doc['rfile_text'] = rfile_texts[i]      # text 담음.
        doc['dense_vectors'] = emb

        docs.append(doc)
        #---------------------------------------------------    

        # Faiss index 생성하고 추가 
        #index = fassi_index(embeddings=emb, method=faiss_index_method)
        #faissindexlist.append(index)

        if count % BATCH_SIZE == 0:
            index_batch(es, docs, vector_len=num_clusters, dim_size=dimension)
            docs = []
            print("Indexed {} documents.".format(count))

    if docs:
        index_batch(es, docs, vector_len=num_clusters, dim_size=dimension)
        print("Indexed {} documents.".format(count))   

    es.indices.refresh(index=INDEX_NAME)

    print(f'*인덱싱 시간 : {time.time()-start:.4f}\n')


In [ ]:
#======================================================================================
# ElasticSearch(이하:ES) 데이터 인텍싱
# - ElasticSearch(이하:ES)에 KorQuAD_v1.0_train_convert.json 파일에 vector값을 구하여 index 함
#
# => index 명 : korquad
# => index 구조 : index_1.json 파일 참조
# => BATCH_SIZE : 100 => 100개의 vector값을 구하여, 한꺼번에 ES에 인텍스 데이터를 추가함.
#======================================================================================
INDEX_NAME = 'mpower_128d_10_float16'  # ES 인덱스 명 (*소문자로만 지정해야 함)
INDEX_FILE = './data/mpower10u_128d_10.json'                 # 인덱스 구조 파일
BATCH_SIZE = 100

# 1. elasticsearch 접속
es = Elasticsearch("http://192.168.0.27:9200/")
print(es.info())

create_index(es, True)

# 2. index 처리
index_data()

In [ ]:
#=========================================================
# 여기서 부터는 문서 검색 처리임.
#=========================================================

In [ ]:
#-----------------------------------------------------------
# 검색
#-----------------------------------------------------------
from elasticsearch import Elasticsearch
from elasticsearch import helpers

#-------------------------
# param
#-------------------------

query_num = 0               # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
INDEX_NAME = 'mpower-kpf-128d-f16-variable'  # ES 인덱스 명 (*소문자로만 지정해야 함)
SEARCH_SIZE = 5             # 검색 계수

# ES 벡터 크기 값(임의이 값지정) =>벡터의 크기는 각 구성 요소의 제곱 합의 제곱근으로 정의된다.. 
# 예를 들어, 벡터 [1, 2, 3]의 크기는 sqrt(1^2 + 2^2 + 3^2) 즉, 3.7416이 된다.
# 클수록 -> 스코어는 작아짐, 작을수록 -> 스코어 커짐.
VECTOR_MAG = 0.8   
SEARCH_RESULT_OUT_TXT = 'sresult.txt'  # 검색결과 쿼리와 스코어를 저장하는 파일명

#-------------------------

# elastic 서버 접속 
#es = Elasticsearch("https://192.168.0.91:9200/", verify_certs=False)
es = Elasticsearch("http://10.10.4.10:9200/")
es.info()

In [ ]:
#------------------------------------------------------------
# 쿼리 df 만듬.
# => 인덱스내 데이터 조회 => query 이용해서 데이터 조회 후 쿼리 df 만듬
# 
# GET /index명/_search
#{
#  "_source": ["rfile_name","rfile_text"], 
#  "query": {
#    "match_all": {}
#  }
# }	
#-----------------------------------------------------------
import pandas as pd

def search(index_name, data=None, source:list=None):
    
    if data is None: #모든 데이터 조회
        data = {"match_all":{}}
    else:
        data = {"match": data}
    
    if source is None:
        body = {"query": data}
    else:
        body = {"_source":source, "query": data}
    
    #print(body)
    
    res = es.search(index=index_name, body=body)
    return res

# 쿼리로 rfile_name 1001 부터 1252까지 쿼리하면서 rfile_name과 rfile_text 불러옴.
rfile_list = []

for i in range(253):
    contextid = 1000+i
    data = {'rfile_name': contextid}
    res=search(index_name=INDEX_NAME, data=data, source=["rfile_name","rfile_text"])

    for hits in res['hits']['hits']:
        rfile_name = hits['_source']['rfile_name']
        rfile_text = hits['_source']['rfile_text']
        
        if rfile_name and rfile_text:
            docs = {}
            docs['rfile_name'] = rfile_name
            docs['rfile_text'] = rfile_text
            
        rfile_list.append(docs)
        break

# 리스트를 불러와서 질의 dataframe 만듬
contextids = []
questions = []

for i, rfile in enumerate(rfile_list):
    rfile_name = rfile['rfile_name']
    rfile_text = rfile['rfile_text']
    
    contextids.append(rfile_name)
    questions.append(rfile_text)
    
    if i < 10:
        print(f'{rfile_name} : {rfile_text}')
 
# dataframe으로 만듬
df_questions = pd.DataFrame((zip(questions, contextids)), columns = ['question','contextid'])

In [ ]:
df_questions.tail()

In [ ]:
'''
#----------------------------------------------------------
# 질의 데이터 불러옴.
#----------------------------------------------------------
import pandas as pd
import json

filepath = '../../data11/mpower_doc/mquestion.json'

# json파일 불러오기
with open(filepath, 'r', encoding='UTF-8') as f:
    json_data = json.load(f)
df_questions = pd.DataFrame(json_data)

print(df_questions.tail())
print()
print(df_questions['contextid'][0:5])
print(df_questions['question'][0:5])
'''

In [ ]:
#-------------------------------------------------------------------------------------
# 쿼리문장 샘플링 후 임베딩 처리함
#user_querys = ["독도에서 사고가 나서 실종자가 발생했다.", "오늘 날씨가 흐리고 비가 오겠다."]
#-------------------------------------------------------------------------------------
import time
from myutils import df_sampling, make_query_script

#--------------------------------------------------
# 쿼리 샘플링함.
if query_num == 0:   # query_num = 0 이면 모든 쿼리
    user_querys = df_questions['question'].values.tolist()
else:   # query_num > 0이면 해당 계수만큼 랜덤하게 샘플링하여 쿼리 목록을 만듬.
    df_questions = df_sampling(df=df_questions, num=query_num, seed=seed)
    user_querys = df_questions['question'].values.tolist()
  
print(f'Query-----------------------------------------------------')
print(user_querys[0:4])
print()

#--------------------------------------------------
# 쿼리들에 대해 임베딩 값 구함
start_embedding_time = time.time()

embed_querys = embedding(user_querys)

end_embedding_time = time.time() - start_embedding_time
print("embedding time: {:.2f} ms".format(end_embedding_time * 1000)) 
print(f'*embed_querys.shape:{embed_querys.shape}\n')

In [ ]:
user_querys[2]

In [ ]:
embed_querys[2]

In [ ]:
#-------------------------------------------------------------------------------------
# 쿼리 스크립트 구성후, ES로 쿼리 날림.
#-------------------------------------------------------------------------------------
from myutils import make_query_script

bi_predictions_list = []
start_search_time = time.time()
    
 # 검색 결과를 파일로 저장
with open(SEARCH_RESULT_OUT_TXT, 'w', encoding='utf-8') as f:
        
    for i, embed_query in enumerate(embed_querys):
        script_query = make_query_script(query_vector=embed_query, vectormag=VECTOR_MAG, vectornum=10) # 쿼리를 만듬.

        # 실제 ES로 검색 쿼리 날림
        response = es.search(
            index=INDEX_NAME,
            body={
                "size": SEARCH_SIZE * 3,  # 일단 3배 쿼리해둠.
                "query": script_query,
                "_source":{"includes": ["rfile_name","rfile_text"]}
            }
        )

        #if i < 5:
        #    print("{} total hits.\n".format(response["hits"]["total"]["value"])) 

        # 쿼리 응답 결과값에서 _id, _score, _source 등을 뽑아냄
        #print(response)
        rfilename = []
        rfiletext = [] 
        bi_scores = []
        for hit in response["hits"]["hits"]: 
            tmp = hit["_source"]["rfile_name"]
            # 중복 제거
            if tmp and tmp not in rfilename:
                rfilename.append(hit["_source"]["rfile_name"])
                rfiletext.append(hit["_source"]["rfile_text"])
                bi_scores.append(hit["_score"])

        if i < 3:
            print(f'bi_scores: {bi_scores}')
            print(f'frilename: {rfilename}')
        
         # 내림 차순으로 정렬
        dec_bi_scores = reversed(np.argsort(bi_scores))
        #if i < 3:
        #    print(dec_bi_scores)
        #    print()

        # 내림차순으로 출력
        #tmp_bi_predictions_list = []
        #tmp_bi_predictions_list.append(rfilename)

        # 파일로 쿼리와 결과 값 저장
        writequery = "[{}] {}".format(i, user_querys[i])
        if i < 5:
            print(writequery)
            
        f.write('\n'+writequery+'\n')

        for idx in dec_bi_scores:
            writetext = "{:.2f}\t[contextid:{}] {}".format(float(bi_scores[idx]), rfilename[idx], rfiletext[idx])
            f.write(writetext+'\n')
            
            if i < 5:
                print(writetext)


        # 2D 예측검색결과 리스트에 추가 
        bi_predictions_list.append(rfilename[0:SEARCH_SIZE])

    end_search_time = time.time() - start_search_time
    print("*검색시간: {:.2f} ms".format(end_search_time * 1000)) 


In [ ]:
#--------------------------------------------------------------------------------------------------
# 6. MRR 계산
# => 정답 리스트[2,3,1,4] 과 예측검색리스트[[1,2,5,1],[3,4,2,1],[6,5,4,1], [2,3,4,1]]를 입력하여 MRR 스코어 구함
##--------------------------------------------------------------------------------------------------
from myutils import mean_reciprocal_rank

# 정답, 여기서는 contextid를 리스트로 만듬.
ground_truths_list = df_questions['contextid'].values.tolist()
#print(f'gtlen:{len(ground_truths_list)}')
#print(ground_truths_list[0:9])

logger.info(f'---Parameter------------------------------------------------------------')
logger.info(f'쿼리/인덱싱 : EMBEDDING_METHOD={EMBEDDING_METHOD}(0=문장클러스터링, 1=문장평균임베딩, 2=문장임베딩), FLOAT_TYPE={FLOAT_TYPE}, seed={seed}, query_num={query_num}, search_k={search_k}, avg_num={avg_num}, faiss_index_method={faiss_index_method}')
logger.info(f'슬라이딩 윈도우 : IS_SLIDING_WINDOW={IS_SLIDING_WINDOW}, WINDOW_SIZE={WINDOW_SIZE}, SLIDING_SIZE={SLIDING_SIZE}')
logger.info(f'클러스터링 : CLUSTRING_MODE={CLUSTRING_MODE}, num_clusters={num_clusters}, outmode={outmode}')
logger.info(f'ONNX: IS_ONNX_MODEL={IS_ONNX_MODEL}')
logger.info(f'차원 축소: out_dimension={out_dimension}')
logger.info(f'전처리 : remove_sentence_len={remove_sentence_len}, remove_duplication={remove_duplication}')

# MRR 계산
bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, bi_predictions_list)

# BI-MRR 출력
logger.info(f'----------------------------------------------------------------------------')
logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
logger.info('*BI-MRR:{:.4f}'.format(bi_score))
logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:10]}')

# 10개씩 출력해봄.
if len(bi_ranks) > 10:
    print()
    print(f'BI_RANKS 10개씩 출력')
    print('------------------------------------------------------------------------------')
    subarrays = [bi_ranks[i:i+10] for i in range(0, len(bi_ranks), 10)]
    # Print the resulting subarrays
    for i, subarray in enumerate(subarrays):
        print(f"{i}: {subarray}")
    
# 검색 한 계슈
#logger.info(f'---------------------------------------------------------------------------')
search_count = 0
nosearch_count = 0
nosearch_list = []
for i,item in enumerate(bi_ranks):
    if item != 0:
        search_count += 1
    else:
        nosearch_count += 1
        nosearch_list.append(i)
    
logger.info('*검색률: {}/{}({:.2f}%)'.format(search_count, len(bi_ranks), (search_count/len(bi_ranks))*100))
logger.info(f'---------------------------------------------------------------------------')

print()
print('*검색실패 : {}'.format(nosearch_count))
for i, nosearch in enumerate(nosearch_list):
    print(f'[{nosearch}] : {df_questions["question"][nosearch]}')